In [ ]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import (
    Input, Conv2D, MaxPooling2D, Flatten, Dense, Dropout
)
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from sklearn.metrics import f1_score, roc_auc_score
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import load_img, img_to_array

# --------------------------- Configuration ---------------------------
train_path = '/kaggle/input/ucf-crime-dataset/Train'
test_path = '/kaggle/input/ucf-crime-dataset/Test'
batch_size = 32
img_size = (128, 128)  # Reduced from 240x240 for efficiency
num_classes = 10  # Update with actual class count

# --------------------------- Image Data Generator ---------------------------
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import (
    Input, Conv2D, MaxPooling2D, Flatten, Dense, Dropout
)
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from sklearn.metrics import f1_score, roc_auc_score
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import load_img, img_to_array

# --------------------------- Configuration ---------------------------
train_path = '/kaggle/input/ucf-crime-dataset/Train'
test_path = '/kaggle/input/ucf-crime-dataset/Test'
batch_size = 32
img_size = (128, 128)
num_classes = 10
images_per_class = 20  # Added parameter

# --------------------------- Modified Image Data Generator ---------------------------
class ImageDataGenerator(tf.keras.utils.Sequence):
    def __init__(self, base_path, batch_size, img_size, num_classes):
        self.batch_size = batch_size
        self.img_size = img_size
        self.num_classes = num_classes
        
        # Collect image paths and labels
        self.image_paths = []
        self.labels = []
        
        # Get class folders
        self.classes = sorted([
            d for d in os.listdir(base_path) 
            if os.path.isdir(os.path.join(base_path, d))
        ])[:num_classes]

        for class_idx, class_name in enumerate(self.classes):
            class_path = os.path.join(base_path, class_name)
            
            # Get first 20 images in class directory
            images = [
                os.path.join(class_path, f) 
                for f in os.listdir(class_path) 
                if f.lower().endswith(('.png', '.jpg', '.jpeg'))
            ][:images_per_class]  # Modified line
            
            self.image_paths.extend(images)
            self.labels.extend([class_idx] * len(images))

        # Shuffle data
        self.indices = np.arange(len(self.image_paths))
        np.random.shuffle(self.indices)

    # Rest of the class remains the same
    # ... [keep other methods unchanged]

# --------------------------- Rest of the code remains identical ---------------------------
# ... [keep model architecture, training, and evaluation the same]
    def __len__(self):
        return int(np.ceil(len(self.image_paths) / self.batch_size))

    def __getitem__(self, index):
        batch_indices = self.indices[
            index*self.batch_size : (index+1)*self.batch_size
        ]
        
        batch_images = []
        batch_labels = []
        
        for i in batch_indices:
            img = load_img(self.image_paths[i], target_size=self.img_size)
            img_array = img_to_array(img) / 255.0
            batch_images.append(img_array)
            batch_labels.append(self.labels[i])
        
        return np.array(batch_images), tf.keras.utils.to_categorical(batch_labels, num_classes=self.num_classes)

# --------------------------- CNN Model Architecture ---------------------------
def build_image_model(input_shape, num_classes):
    inputs = Input(shape=input_shape)
    
    x = Conv2D(32, (3, 3), activation='relu')(inputs)
    x = MaxPooling2D((2, 2))(x)
    x = Conv2D(64, (3, 3), activation='relu')(x)
    x = MaxPooling2D((2, 2))(x)
    x = Flatten()(x)
    x = Dense(128, activation='relu')(x)
    x = Dropout(0.5)(x)
    outputs = Dense(num_classes, activation='softmax')(x)
    
    model = Model(inputs, outputs)
    model.compile(
        optimizer=Adam(learning_rate=0.001),
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )
    return model

# --------------------------- Execution ---------------------------
# Initialize generators
train_gen = ImageDataGenerator(train_path, batch_size, img_size, num_classes)
test_gen = ImageDataGenerator(test_path, batch_size, img_size, num_classes)

# Build model
model = build_image_model((*img_size, 3), num_classes)
model.summary()

# Callbacks
callbacks = [
    EarlyStopping(patience=5, restore_best_weights=True),
    ReduceLROnPlateau(factor=0.5, patience=3)
]

# Training
history = model.fit(
    train_gen,
    validation_data=test_gen,
    epochs=30,
    callbacks=callbacks,
    verbose=1
)

# --------------------------- Evaluation ---------------------------
loss, accuracy = model.evaluate(test_gen)
print(f"Test Accuracy: {accuracy * 100:.2f}%")

# Predictions
y_pred = model.predict(test_gen)
y_true = np.concatenate([np.argmax(test_gen[i][1], axis=1) for i in range(len(test_gen))])

# Metrics
print(f"F1 Score: {f1_score(y_true, np.argmax(y_pred, axis=1), average='macro'):.4f}")
print(f"AUC Score: {roc_auc_score(tf.keras.utils.to_categorical(y_true, num_classes), y_pred, multi_class='ovr'):.4f}")

# Plots
plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Train')
plt.plot(history.history['val_accuracy'], label='Validation')
plt.title('Accuracy')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Train')
plt.plot(history.history['val_loss'], label='Validation')
plt.title('Loss')
plt.legend()
plt.show()